In [41]:
GIGACHAT_DIR = "..."
DEEPSEEK_DIR = "..."
DEVICE = "cuda:0"
NUM_LAYERS = 61

RESULT_CSV_PATH = "..."

In [30]:
import os, re, glob, time
import torch
import pandas as pd
import numpy as np
from contextlib import ExitStack
from typing import Optional, Sequence
from safetensors import safe_open

# ---------- regex setup ----------
DEFAULT_PREFIXES = (
    r"model\.layers\.(\d+)\.",       # llama/qwen/deepseek-style
)

# --- skip patterns (ignore at indexing/load stage) ---
DEFAULT_SKIP_REGEXES = (
    r"\.experts\.",   # ignore MoE experts tensors
)

def iter_safetensors_files(model_dir):
    files = sorted(glob.glob(os.path.join(model_dir, "*.safetensors")))
    if not files:
        raise FileNotFoundError(f"No .safetensors files in {model_dir}")
    return files

def compile_regexes(patterns):
    if not patterns:
        return []
    return [re.compile(p) for p in patterns]

def compile_prefixes(prefixes):
    return compile_regexes(prefixes)

def _should_skip_key(key, regs):
    for rgx in regs:
        if rgx.search(key):
            return True
    return False

def extract_layer_id(key, regs):
    for rgx in regs:
        m = rgx.search(key)
        if m:
            return int(m.group(1))
    return None

# ---------- 1) lightweight index: layer -> {key: filepath} ----------
def build_layer_index(
    model_dir,
    n_layers=3,
    prefixes=DEFAULT_PREFIXES,
    skip_regexes=DEFAULT_SKIP_REGEXES,   # <-- default ignore here
):
    files = iter_safetensors_files(model_dir)
    regs = compile_prefixes(prefixes)
    skip_regs = compile_regexes(skip_regexes)

    layer_to_keys = {i: {} for i in range(n_layers)}
    for path in files:
        with safe_open(path, framework="pt", device="cpu") as f:
            for k in f.keys():
                if skip_regs and _should_skip_key(k, skip_regs):
                    continue
                lid = extract_layer_id(k, regs)
                if lid is None or lid >= n_layers:
                    continue
                layer_to_keys[lid][k] = path
    return layer_to_keys

# ---------- 2) metrics ----------
def _cos_sim(fa, fb, eps=1e-12):
    denom = (fa.norm() * fb.norm()).clamp_min(eps)
    return (fa @ fb / denom).item()

def _pearson(fa, fb, eps=1e-12):
    fa_c = fa - fa.mean()
    fb_c = fb - fb.mean()
    denom = (fa_c.norm() * fb_c.norm()).clamp_min(eps)
    return (fa_c @ fb_c / denom).item()

def _spearman(fa, fb, eps=1e-12):
    ra = torch.argsort(torch.argsort(fa))
    rb = torch.argsort(torch.argsort(fb))
    ra = ra.float() - ra.float().mean()
    rb = rb.float() - rb.float().mean()
    denom = (ra.norm() * rb.norm()).clamp_min(eps)
    return (ra @ rb / denom).item()

def _quantiles(abs_diff, qs=(0.01, 0.05, 0.5, 0.95, 0.99), max_samples=None):
    if abs_diff.numel() == 0:
        return {}

    vals = abs_diff.flatten()

    if max_samples is not None and vals.numel() > max_samples:
        idx = torch.randperm(vals.numel(), device=vals.device)[:max_samples]
        vals = vals[idx]

    qv = torch.tensor(qs, device=vals.device, dtype=vals.dtype)
    try:
        out = torch.quantile(vals, qv)
    except RuntimeError:
        vals_cpu = vals.float().cpu().numpy()
        qv_np = qv.float().cpu().numpy()
        out_np = np.quantile(vals_cpu, qv_np)
        out = torch.from_numpy(out_np).to(vals)

    return {f"q{int(q*100):02d}_abs_diff": out[i].item() for i, q in enumerate(qs)}

def _sample_pair(fa, fb, max_samples=None):
    if max_samples is None or fa.numel() <= max_samples:
        return fa, fb
    idx = torch.randint(0, fa.numel(), (max_samples,), device=fa.device)
    return fa[idx], fb[idx]

def _rel_l2_err(fa, fb, eps=1e-12):
    return (fa - fb).norm().div(fa.norm().clamp_min(eps)).item()

def _rel_max_abs_diff(fa, fb, eps=1e-12):
    denom = fa.abs().clamp_min(eps)
    return ((fa - fb).abs() / denom).max().item()

def _rel_mean_abs_diff(fa, fb, eps=1e-12):
    denom = fa.abs().clamp_min(eps)
    return ((fa - fb).abs() / denom).mean().item()


# ---------- 3) streaming compare per layer ----------
def compare_models_streaming(
    gigachat_dir,
    deepseek_dir,
    n_layers=3,
    prefixes=DEFAULT_PREFIXES,
    cast_to=torch.float32,
    device="cpu",
    do_spearman=True,
    quantile_max_samples: Optional[int] = None,
    spearman_max_samples: Optional[int] = None,
    log_every=5,
    max_tensors=None,
    verbose=True,
    skip_regexes=DEFAULT_SKIP_REGEXES,  # <-- default ignore here too
):
    # lightweight indexes
    gc_index = build_layer_index(
        gigachat_dir,
        n_layers=n_layers,
        prefixes=prefixes,
        skip_regexes=skip_regexes,
    )
    ds_index = build_layer_index(
        deepseek_dir,
        n_layers=n_layers,
        prefixes=prefixes,
        skip_regexes=skip_regexes,
    )

    rows = []
    processed = 0
    interrupted = False
    start_time = time.time()

    stop_requested = False

    try:
        for lid in range(n_layers):
            gc_keys = gc_index[lid]
            ds_keys = ds_index[lid]
            common_keys = sorted(set(gc_keys) & set(ds_keys))

            if verbose:
                print(
                    f"[{time.strftime('%H:%M:%S')}] layer {lid} "
                    f"({len(common_keys)} common tensors)"
                )

            if not common_keys:
                continue

            layer_start = time.time()

            with ExitStack() as stack:
                gc_handles = {}
                ds_handles = {}

                def get_tensor(handles, path, key):
                    if path not in handles:
                        handles[path] = stack.enter_context(
                            safe_open(path, framework="pt", device=str(device))
                        )
                    return handles[path].get_tensor(key)

                for idx, k in enumerate(common_keys, 1):
                    a = get_tensor(gc_handles, gc_keys[k], k).to(dtype=cast_to)
                    b = get_tensor(ds_handles, ds_keys[k], k).to(dtype=cast_to)

                    if a.shape != b.shape:
                        rows.append(dict(
                            layer=lid, key=k,
                            shape_a=tuple(a.shape), shape_b=tuple(b.shape),
                            note="shape_mismatch",
                        ))
                        processed += 1
                        continue

                    fa = a.reshape(-1)
                    fb = b.reshape(-1)
                    abs_diff = (fa - fb).abs()

                    row = dict(
                        layer=lid,
                        key=k,
                        shape=tuple(a.shape),
                        numel=int(fa.numel()),
                        norm_a=fa.norm().item(),
                        norm_b=fb.norm().item(),
                        norm_ratio=(fa.norm() / fb.norm().clamp_min(1e-12)).item(),
                        mean_a=fa.mean().item(),
                        mean_b=fb.mean().item(),
                        mean_ratio=(fa.mean() / fb.mean().clamp_min(1e-12)).item(),
                        std_a=fa.std(unbiased=False).item(),
                        std_b=fb.std(unbiased=False).item(),
                        std_ratio=(fa.std(unbiased=False) / fb.std(unbiased=False).clamp_min(1e-12)).item(),
                        zero_frac_a=(fa == 0).float().mean().item(),
                        zero_frac_b=(fb == 0).float().mean().item(),
                        cos_sim=_cos_sim(fa, fb),
                        rel_l2_err=_rel_l2_err(fa, fb),
                        mean_rel_diff=_rel_mean_abs_diff(fa, fb),
                        max_rel_diff=_rel_max_abs_diff(fa, fb),
                        pearson=_pearson(fa, fb),
                        max_abs_diff=abs_diff.max().item(),
                        mean_abs_diff=abs_diff.mean().item(),
                    )

                    if do_spearman:
                        fa_sp, fb_sp = _sample_pair(fa, fb, spearman_max_samples)
                        row["spearman"] = _spearman(fa_sp, fb_sp)

                    row.update(_quantiles(abs_diff, max_samples=quantile_max_samples))

                    rows.append(row)
                    processed += 1

                    if verbose and (processed == 1 or processed % max(1, log_every) == 0):
                        elapsed = time.time() - start_time
                        rate = processed / max(elapsed, 1e-6)
                        layer_elapsed = time.time() - layer_start
                        print(
                            f"[{time.strftime('%H:%M:%S')}] processed "
                            f"{processed} tensors ({rate:.1f}/s, "
                            f"layer {lid} {idx}/{len(common_keys)} in {layer_elapsed:.1f}s)"
                        )

                    if max_tensors is not None and processed >= max_tensors:
                        stop_requested = True
                        break

                if stop_requested:
                    break
    except KeyboardInterrupt:
        interrupted = True
        if verbose:
            print(
                f"[{time.strftime('%H:%M:%S')}] KeyboardInterrupt received, "
                "returning partial results."
            )

    total_elapsed = time.time() - start_time
    if verbose:
        status = "interrupted" if interrupted else "finished"
        print(
            f"[{time.strftime('%H:%M:%S')}] {status}: {processed} tensors "
            f"in {total_elapsed:.1f}s ({processed / max(total_elapsed, 1e-6):.1f}/s)"
        )

    df = pd.DataFrame(rows)
    df.attrs["interrupted"] = interrupted
    df.attrs["processed_tensors"] = processed
    df.attrs["elapsed_seconds"] = total_elapsed
    df.attrs["stop_requested"] = stop_requested
    return df

# ---------- 4) usage ----------
df = compare_models_streaming(
    gigachat_dir=GIGACHAT_DIR,
    deepseek_dir=DEEPSEEK_DIR,
    n_layers=NUM_LAYERS,
    device="cuda:0",
    
)

[12:06:51] layer 0 (12 common tensors)
[12:06:51] processed 1 tensors (158.6/s, layer 0 1/12 in 0.0s)
[12:06:58] processed 5 tensors (0.6/s, layer 0 5/12 in 7.7s)
[12:06:58] processed 10 tensors (1.3/s, layer 0 10/12 in 7.8s)
[12:06:58] layer 1 (12 common tensors)
[12:07:03] processed 15 tensors (1.2/s, layer 1 3/12 in 4.8s)
[12:07:06] layer 2 (12 common tensors)
[12:07:06] processed 25 tensors (1.6/s, layer 2 1/12 in 0.0s)
[12:07:13] processed 30 tensors (1.3/s, layer 2 6/12 in 7.1s)
[12:07:13] processed 35 tensors (1.6/s, layer 2 11/12 in 7.2s)
[12:07:13] layer 3 (14 common tensors)
[12:07:13] processed 40 tensors (1.8/s, layer 3 4/14 in 0.0s)
[12:07:13] processed 45 tensors (2.0/s, layer 3 9/14 in 0.1s)
[12:07:13] layer 4 (14 common tensors)
[12:07:13] processed 55 tensors (2.4/s, layer 4 5/14 in 0.1s)
[12:07:14] layer 5 (14 common tensors)
[12:07:14] processed 65 tensors (2.8/s, layer 5 1/14 in 0.0s)
[12:07:14] processed 70 tensors (3.0/s, layer 5 6/14 in 0.1s)
[12:07:14] layer 6 (

In [35]:
SUMMARY_METRICS = [
    "cos_sim",
    "pearson",
    "spearman",
    "rel_l2_err",
    "mean_rel_diff",
    "max_rel_diff",
    "mean_abs_diff",
    "max_abs_diff",
    "norm_ratio",
    "std_ratio",
    "mean_ratio",
]

def _safe_filter_columns(df, cols: Sequence[str]):
    return [c for c in cols if c in df.columns]

# --- LayerNorm detection ---
LAYERNORM_SUFFIXES = (
    ".layernorm.weight",
    ".layer_norm.weight",
    ".ln.weight",
    ".ln_f.weight",
    "layernorm.weight",
    "layer_norm.weight",
    # bias here too
    "e_score_correction_bias",
)
def is_layernorm_key(key: str, suffixes=LAYERNORM_SUFFIXES) -> bool:
    return any(key.endswith(suf) for suf in suffixes)

def _describe(df_part: pd.DataFrame, metrics: Sequence[str]):
    if df_part is None or len(df_part) == 0:
        return pd.DataFrame()
    return df_part[list(metrics)].describe(
        percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]
    )

def _closeness_stats(df_part: pd.DataFrame, prefix=""):
    out = {}
    if df_part is None or len(df_part) == 0:
        return out
    if "cos_sim" in df_part:
        out[f"{prefix}cos_gt_0.99_frac"] = (df_part["cos_sim"] > 0.99).mean()
        out[f"{prefix}cos_gt_0.999_frac"] = (df_part["cos_sim"] > 0.999).mean()
    return out
    
def summarize_results(df: pd.DataFrame):
    # drop shape mismatches and other non-comparable rows
    comparable = df[df.get("note").isna()]
    shape_mismatches = df["note"].eq("shape_mismatch").sum() if "note" in df else 0

    metrics = _safe_filter_columns(comparable, SUMMARY_METRICS)

    # split layernorm vs others
    if "key" in comparable:
        ln_mask = comparable["key"].apply(is_layernorm_key)
    else:
        ln_mask = pd.Series(False, index=comparable.index)

    ln_part = comparable[ln_mask]
    other_part = comparable[~ln_mask]

    ln_summary = _describe(ln_part, metrics)
    other_summary = _describe(other_part, metrics)

    layer_summary = {}
    layer_summary_ln = {}
    layer_summary_other = {}
    if "layer" in comparable:
        if len(comparable) > 0:
            layer_summary = comparable.groupby("layer")[metrics].mean()
        if len(ln_part) > 0:
            layer_summary_ln = ln_part.groupby("layer")[metrics].mean()
        if len(other_part) > 0:
            layer_summary_other = other_part.groupby("layer")[metrics].mean()

    closeness = {}
    closeness.update(_closeness_stats(comparable, prefix="all_"))
    closeness.update(_closeness_stats(ln_part, prefix="ln_"))
    closeness.update(_closeness_stats(other_part, prefix="other_"))

    coverage = {
        "total_rows": len(df),
        "comparable_rows": len(comparable),
        "shape_mismatches": int(shape_mismatches),
        "layernorm_rows": int(len(ln_part)),
        "other_rows": int(len(other_part)),
    }

    return {
        "coverage": coverage,
        "closeness": closeness,
        "layer_mean": layer_summary,
        "layernorm_summary": ln_summary,
        "other_summary": other_summary,
        "layernorm_layer_mean": layer_summary_ln,
        "other_layer_mean": layer_summary_other,
    }

def print_summary(summary):
    if isinstance(summary.get("other_summary"), pd.DataFrame) and not summary["other_summary"].empty:
        print("\n=== non-layernorm stats (describe) ===")
        print(summary["other_summary"])

    if isinstance(summary.get("other_layer_mean"), pd.DataFrame) and not summary["other_layer_mean"].empty:
        print("\n=== per-layer mean metrics (non-layernorm) ===")
        print(summary["other_layer_mean"])

    if isinstance(summary.get("layernorm_summary"), pd.DataFrame) and not summary["layernorm_summary"].empty:
        print("\n=== layernorm stats (describe) ===")
        print(summary["layernorm_summary"])

summary = summarize_results(df)
print_summary(summary)


=== non-layernorm stats (describe) ===
          cos_sim     pearson    spearman    rel_l2_err  mean_rel_diff  \
count  363.000000  363.000000  363.000000    363.000000     363.000000   
mean    -0.000037   -0.000040   -0.000002   4294.096856   57084.277631   
std      0.001984    0.001982    0.001965   2463.475244   43614.327872   
min     -0.009123   -0.009121   -0.008588      1.144950       4.780324   
1%      -0.006066   -0.006074   -0.006189      1.159569       5.800548   
5%      -0.003547   -0.003558   -0.003408      1.181075       6.962997   
50%      0.000011    0.000011    0.000010   4663.833496   53996.414062   
95%      0.003594    0.003576    0.003543   7572.696387  123171.325000   
99%      0.006864    0.006845    0.006718  11331.787813  209752.977500   
max      0.007981    0.007958    0.008460  16530.933594  326736.812500   

       max_rel_diff  mean_abs_diff  max_abs_diff  norm_ratio   std_ratio  \
count  3.630000e+02     363.000000    363.000000  363.000000  363.000

In [36]:
pd.set_option('display.max_rows', None)        # все строки
pd.set_option('display.max_columns', None)     # все колонки
pd.set_option('display.width', None)           # не ограничивать ширину вывода
pd.set_option('display.max_colwidth', None)    # не обрезать содержимое ячеек

ln_mask = df["key"].str.endswith(LAYERNORM_SUFFIXES, na=False)
df_ln = df[ln_mask]
df_nonln = df[~ln_mask]
df_nonln

,layer,key,shape,numel,norm_a,norm_b,norm_ratio,mean_a,mean_b,mean_ratio,std_a,std_b,std_ratio,zero_frac_a,zero_frac_b,cos_sim,rel_l2_err,mean_rel_diff,max_rel_diff,pearson,max_abs_diff,mean_abs_diff,spearman,q01_abs_diff,q05_abs_diff,q50_abs_diff,q95_abs_diff,q99_abs_diff,shape_a,shape_b,note
1,0,model.layers.0.mlp.down_proj.weight,"(7168, 18432)",132120576.0,59.667873,428978.468750,0.000139,1.938400e-07,-5.042707e-03,1.938400e+05,0.005191,37.320744,0.000139,0.0,0.000047,0.000037,7189.438477,83933.062500,6.245747e+11,0.000037,448.040527,23.491154,-6.912752e-06,0.207336,1.003906,13.997543,80.000687,143.996338,NaN,NaN,NaN
2,0,model.layers.0.mlp.gate_proj.weight,"(18432, 7168)",132120576.0,56.593945,233500.296875,0.000242,-1.141391e-05,2.023005e-01,-5.642058e-05,0.004924,20.313309,0.000242,0.0,0.000274,-0.000151,4125.888672,94711.351562,1.379779e+12,-0.000128,448.040039,6.796145,-2.233854e-03,0.035805,0.186165,2.499613,23.998825,79.999825,NaN,NaN,NaN
3,0,model.layers.0.mlp.up_proj.weight,"(18432, 7168)",132120576.0,51.375027,589028.500000,0.000087,4.222969e-07,-3.271475e-03,4.222969e+05,0.004470,51.244949,0.000087,0.0,0.000026,-0.000017,11465.268555,144081.312500,6.944284e+11,-0.000017,448.027222,35.005745,1.957554e-05,0.376633,1.876915,23.983521,111.997208,176.000168,NaN,NaN,NaN
6,0,model.layers.0.self_attn.kv_a_proj_with_mqa.weight,"(576, 7168)",4128768.0,42.883209,143063.109375,0.000300,8.415574e-06,-5.285055e-02,8.415574e+06,0.021105,70.407242,0.000300,0.0,0.000023,-0.000718,3336.111328,27890.207031,5.200393e+09,-0.000717,448.052734,47.079998,-8.733455e-04,0.450989,2.255432,27.993683,160.000259,239.989075,NaN,NaN,NaN
7,0,model.layers.0.self_attn.kv_b_proj.weight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(20480, 512)","(32768, 512)",shape_mismatch
8,0,model.layers.0.self_attn.o_proj.weight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(7168, 12288)","(7168, 16384)",shape_mismatch
10,0,model.layers.0.self_attn.q_a_proj.weight,"(1536, 7168)",11010048.0,32.806965,295782.718750,0.000111,1.081290e-06,-1.848673e-02,1.081290e+06,0.009887,89.141144,0.000111,0.0,0.000010,0.000173,9015.852539,108942.070312,8.457782e+10,0.000173,448.038086,68.106529,-2.073852e-05,0.940406,4.515320,52.008484,176.012634,255.995758,NaN,NaN,NaN
11,0,model.layers.0.self_attn.q_b_proj.weight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(12288, 1536)","(24576, 1536)",shape_mismatch
13,1,model.layers.1.mlp.down_proj.weight,"(7168, 18432)",132120576.0,31.056589,311506.968750,0.000100,1.974590e-08,-6.014036e-03,1.974590e+04,0.002702,27.100826,0.000100,0.0,0.000067,-0.000085,10030.302734,92496.351562,2.210074e+11,-0.000085,448.031982,15.785647,-8.367174e-05,0.142975,0.748177,9.001137,52.000927,104.000656,NaN,NaN,NaN
14,1,model.layers.1.mlp.gate_proj.weight,"(18432, 7168)",132120576.0,46.198711,275344.031250,0.000168,7.893201e-06,2.129500e-01,3.706598e-05,0.004019,23.953739,0.000168,0.0,0.000473,-0.002543,5959.996582,80385.343750,1.901858e+11,-0.002561,448.058594,5.853664,-4.157898e-03,0.020668,0.102169,1.375315,17.999088,111.999542,NaN,NaN,NaN


In [37]:
df_ln

,layer,key,shape,numel,norm_a,norm_b,norm_ratio,mean_a,mean_b,mean_ratio,std_a,std_b,std_ratio,zero_frac_a,zero_frac_b,cos_sim,rel_l2_err,mean_rel_diff,max_rel_diff,pearson,max_abs_diff,mean_abs_diff,spearman,q01_abs_diff,q05_abs_diff,q50_abs_diff,q95_abs_diff,q99_abs_diff,shape_a,shape_b,note
0,0,model.layers.0.input_layernorm.weight,"(7168,)",7168.0,5.333566,3.708255,1.438295,0.058730,0.041479,1.415882,0.022791,0.014067,1.620160,0.0,0.0,0.883294,0.505120,0.325193,5.045249,0.003609,0.272217,0.021565,-0.007118,0.000244,0.001709,0.016113,0.060791,0.127278,NaN,NaN,NaN
4,0,model.layers.0.post_attention_layernorm.weight,"(7168,)",7168.0,1.600154,1.976582,0.809556,0.016732,0.016968,0.986042,0.008790,0.016035,0.548183,0.0,0.0,0.645940,0.964386,1.760695,4720.670898,0.007855,0.676208,0.009883,0.009918,0.000122,0.000732,0.007568,0.023434,0.040568,NaN,NaN,NaN
5,0,model.layers.0.self_attn.kv_a_layernorm.weight,"(512,)",512.0,0.152460,0.231502,0.658569,0.005553,0.006916,0.802899,0.003816,0.007539,0.506165,0.0,0.0,0.550863,1.277797,2.887821,84.512604,-0.015007,0.068909,0.006171,-0.009046,0.000122,0.000535,0.004852,0.015239,0.021301,NaN,NaN,NaN
9,0,model.layers.0.self_attn.q_a_layernorm.weight,"(1536,)",1536.0,8.237963,17.689760,0.465691,0.193372,0.443633,0.435884,0.082398,0.083180,0.990599,0.0,0.0,0.905126,1.312957,1.929219,23.032129,0.012730,1.400146,0.252936,0.021972,0.013672,0.058105,0.260742,0.419006,0.477258,NaN,NaN,NaN
12,1,model.layers.1.input_layernorm.weight,"(7168,)",7168.0,4.966424,4.025096,1.233865,0.053581,0.044444,1.205580,0.023877,0.016881,1.414466,0.0,0.0,0.853763,0.522459,0.437025,12.734513,-0.000896,0.185303,0.022439,-0.002995,0.000244,0.001465,0.016632,0.064941,0.096353,NaN,NaN,NaN
16,1,model.layers.1.post_attention_layernorm.weight,"(7168,)",7168.0,4.805261,5.418404,0.886841,0.045868,0.048390,0.947874,0.033429,0.041884,0.798134,0.0,0.0,0.607914,0.948953,1.461620,755.537292,-0.008112,0.530029,0.031187,-0.004826,0.000285,0.001953,0.020996,0.080315,0.259021,NaN,NaN,NaN
17,1,model.layers.1.self_attn.kv_a_layernorm.weight,"(512,)",512.0,0.091417,0.262622,0.348093,0.002844,0.008173,0.348004,0.002869,0.008241,0.348181,0.0,0.0,0.498078,2.528081,19.070650,1066.807495,0.004631,0.041565,0.007834,-0.010430,0.000132,0.000500,0.005886,0.020419,0.025437,NaN,NaN,NaN
21,1,model.layers.1.self_attn.q_a_layernorm.weight,"(1536,)",1536.0,5.265309,17.526243,0.300424,0.116436,0.419999,0.277228,0.067022,0.153560,0.436454,0.0,0.0,0.813170,2.581912,5.037098,172.381821,-0.004717,0.845947,0.306409,0.003983,0.009131,0.055420,0.300781,0.593750,0.704395,NaN,NaN,NaN
24,2,model.layers.2.input_layernorm.weight,"(7168,)",7168.0,6.698238,5.042212,1.328432,0.074627,0.056358,1.324173,0.026269,0.019253,1.364385,0.0,0.0,0.894958,0.468261,0.384318,10.901235,0.021790,0.197266,0.028536,0.022656,0.000488,0.002197,0.022949,0.073160,0.113362,NaN,NaN,NaN
28,2,model.layers.2.post_attention_layernorm.weight,"(7168,)",7168.0,5.107561,3.691749,1.383507,0.053593,0.039059,1.372092,0.027699,0.019384,1.428917,0.0,0.0,0.798047,0.607275,0.457887,44.881279,0.011214,0.599915,0.024096,0.028088,0.000244,0.001709,0.018066,0.063089,0.121575,NaN,NaN,NaN


In [40]:
df.to_csv(RESULT_CSV_PATH, index=False)